# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2ee9e59460>
├── 'a' --> tensor([[ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165]])
└── 'x' --> <FastTreeValue 0x7f2ee4e3a910>
    └── 'c' --> tensor([[ 1.2078,  0.3281, -0.4170, -0.3034],
                        [-0.6955, -1.5817, -0.1177,  0.1171],
                        [ 0.4062,  1.2244, -0.8230,  1.8042]])

In [4]:
t.a

tensor([[ 1.0048, -0.6266,  0.9732],
        [ 0.8089,  0.0388, -0.4165]])

In [5]:
%timeit t.a

74.3 ns ± 0.0644 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2ee9e59460>
├── 'a' --> tensor([[-0.5561,  0.7337,  2.9852],
│                   [ 1.1323, -0.0720, -0.6201]])
└── 'x' --> <FastTreeValue 0x7f2ee4e3a910>
    └── 'c' --> tensor([[ 1.2078,  0.3281, -0.4170, -0.3034],
                        [-0.6955, -1.5817, -0.1177,  0.1171],
                        [ 0.4062,  1.2244, -0.8230,  1.8042]])

In [7]:
%timeit t.a = new_value

79.2 ns ± 0.0617 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0048, -0.6266,  0.9732],
               [ 0.8089,  0.0388, -0.4165]]),
    x: Batch(
           c: tensor([[ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0048, -0.6266,  0.9732],
        [ 0.8089,  0.0388, -0.4165]])

In [11]:
%timeit b.a

69.2 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4855,  0.4505, -0.3577],
               [-1.1404, -0.1751,  0.2873]]),
    x: Batch(
           c: tensor([[ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042]]),
       ),
)

In [13]:
%timeit b.a = new_value

545 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

894 ns ± 0.605 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 90 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 829 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 681 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2e0f748dc0>
├── 'a' --> tensor([[[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]],
│           
│                   [[ 1.0048, -0.6266,  0.9732],
│                    [ 0.8089,  0.0388, -0.4165]]])
└── 'x' --> <FastTreeValue 0x7f2ee4e3a430>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 85.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2e1765efd0>
├── 'a' --> tensor([[ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165],
│                   [ 1.0048, -0.6266,  0.9732],
│                   [ 0.8089,  0.0388, -0.4165]])
└── 'x' --> <FastTreeValue 0x7f2e1765eee0>
    └── 'c' --> tensor([[ 1.2078,  0.3281, -0.4170, -0.3034],
                        [-0.6955, -1.5817, -0.1177,  0.1171],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 96.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2078,  0.3281, -0.4170, -0.3034],
                       [-0.6955, -1.5817, -0.1177,  0.1171],
                       [ 0.4062,  1.2244, -0.8230,  1.8042]],
              
                      [[ 1.2078,  0.3281, -0.4170, -0.3034],
                       [-0.6955, -1.5817, -0.1177,  0.1171],
                       [ 0.4062,  1.2244, -0.8230,  1.8042]],
              
                      [[ 1.2078,  0.3281, -0.4170, -0.3034],
                       [-0.6955, -1.5817, -0.1177,  0.1171],
                       [ 0.4062,  1.2244, -0.8230,  1.8042]],
              
                      [[ 1.2078,  0.3281, -0.4170, -0.3034],
                       [-0.6955, -1.5817, -0.1177,  0.1171],
                       [ 0.4062,  1.2244, -0.8230,  1.8042]],
              
                      [[ 1.2078,  0.3281, -0.4170, -0.3034],
                       [-0.6955, -1.5817, -0.1177,  0.1171],
                       [ 0.4062,  1.2244, -0.8230,  1.8042]],

In [26]:
%timeit Batch.stack(batches)

83 µs ± 381 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042],
                      [ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042],
                      [ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042],
                      [ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042],
                      [ 1.2078,  0.3281, -0.4170, -0.3034],
                      [-0.6955, -1.5817, -0.1177,  0.1171],
                      [ 0.4062,  1.2244, -0.8230,  1.8042],
                      [ 1.2078,  0.3281, -0.4170, -0.3034],
                   

In [28]:
%timeit Batch.cat(batches)

154 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

357 µs ± 6.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
